<a href="https://colab.research.google.com/github/barauna-lo/SAMBATimes/blob/main/RFI_Dection_in_Pulsars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdution of the Problem

# Libraries

In [2]:
import numpy as np
from astropy.io import fits
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from tqdm import tqdm
from scipy.optimize import curve_fit
from astropy.time import Time
from astropy.timeseries import LombScargle
from scipy.signal import periodogram
import warnings

In [3]:
#And some useful functions that we will use later
def plotSpectrum(spectra,freqs):
    plt.plot(freqs,np.average(spectra,axis=0))
    plt.xlabel("Frequency [GHz]")
    plt.ylabel("Counts")
    plt.show()
    
def plotLC(spectra,periodogram=False,doScatter=False,plotlim=[]):
    if len(plotlim)>0:
        ind_min=int(plotlim[0]/dt)
        ind_max=int(plotlim[1]/dt)
    else:
        ind_min=0
        ind_max=len(spectra)-1
        
    times=np.linspace(0,(len(spectra)-1)*dt,len(spectra))
    times=times[ind_min:ind_max]
    intensities=np.nanmean(spectra,axis=1)
    intensities=intensities[ind_min:ind_max]
    if doScatter:
        plt.scatter(times,intensities)
    else:
        plt.plot(times,intensities)
    plt.xlabel("Time [s]")
    plt.ylabel("Counts")
    plt.show()
    
    
    if periodogram:
        freq, p_ls = LombScargle(times, np.nan_to_num(intensities)).autopower(minimum_frequency=0.000000001,
                                         maximum_frequency=10,
                                         normalization='psd',
                                         samples_per_peak=50)
        plt.plot(freq,p_ls)
        plt.yscale("log")
        plt.xlabel("Frequency")
        plt.ylabel("Spectral Power")
        plt.show()
        
        return freq, p_ls
    
    
    return times,intensities
    
def getStokesI(data):
    spectra = []
    mjds = []
    phi=0 #reference angle for Q-U reference frame

    #we need to convert the data to total intensity
    #since the data are recorded in circular polarization
    for i in range(len(data)):
        spectrum=[]

        mjds.append(data[i][0])

        #extract polarization parameters
        lcp=np.array(data[i][1][0])
        rcp=np.array(data[i][1][1])
        COS=np.array(data[i][1][2])
        SIN=np.array(data[i][1][3])

        I=lcp+rcp
        Q=np.cos(phi)*COS-np.sin(phi)*SIN
        U=np.sin(phi)*COS+np.cos(phi)*SIN
        V=lcp-rcp   


        spectra.append(I)#append STOKES I (Q=1,U=2,V=3)

    return np.array(spectra)
    
#Define the Gaussian function
def Gauss(x, A, B, C, D):
    y = A*np.exp(-1*B*(x-C)**2)+D
    return y
    
#does a simple bandpass calibration by dividing all data by the average spectrum
def calibrateBandpass(spectra):
    for i in range(len(spectra[0])):
        add_count=0
        for j in range(len(spectra)):
            add_count+=spectra[j][i]
        corr_factor=add_count/len(spectra)
        for j in range(len(spectra)):
            spectra[j][i]=spectra[j][i]/corr_factor
            
    return spectra

def removeNoiseCal(spectra,showPlots=False,removeSource=False):
    
    #remove noise cal effect 
    for j in tqdm(range(len(spectra[0]))):

        #seperate the two/four phases
        bin1=[]
        bin2=[]
        count=2
        switch=True
        for i in range(len(spectra)):
            if count==0:
                switch=not switch 
                count=2
            if switch:
                bin1.append(spectra[i][j])
            elif not switch:
                bin2.append(spectra[i][j])
            count-=1

        if removeSource:
            try:
                parameters1, covariance1 = curve_fit(Gauss, np.arange(len(bin1)), np.array(bin1),p0=np.asarray([0.1,1/100,len(bin1)/2,np.average(bin1)]),maxfev = 2000)
            except:
                parameters1=[0,0,0,0]
            try:    
                parameters2, covariance2 = curve_fit(Gauss, np.arange(len(bin2)), np.array(bin2),p0=np.asarray([0.1,1/100,len(bin2)/2,np.average(bin2)]),maxfev = 2000)
            except:
                parameters2=[0,0,0,0]

            fit_A1 = parameters1[0]
            fit_B1 = parameters1[1]
            fit_C1 = parameters1[2]
            fit_D1 = parameters1[3]
            fit_A2 = parameters2[0]
            fit_B2 = parameters2[1]
            fit_C2 = parameters2[2]
            fit_D2 = parameters2[3]

            fit1 = Gauss(np.arange(len(bin1)), fit_A1, fit_B1, fit_C1, fit_D1)
            fit2 = Gauss(np.arange(len(bin2)), fit_A2, fit_B2, fit_C2, fit_D2)
        else:
            fit1 = np.full(len(bin1),np.average(bin1))
            fit2 = np.full(len(bin2),np.average(bin2))


        if showPlots:
            plt.scatter(np.arange(len(bin1)),bin1)
            plt.plot(np.arange(len(bin1)),fit1,color="yellow")
            plt.scatter(np.arange(len(bin2)),bin2)
            plt.plot(np.arange(len(bin2)),fit2,color="red")
            plt.show()
        
        
        bin1=bin1-fit1
        bin2=bin2-fit2

        final_time=[]
        i=0
        while i<len(bin1):
            final_time.append(bin1[i])
            if not i+1>=len(bin1):
                final_time.append(bin1[i+1])
            if not i>=len(bin2):
                final_time.append(bin2[i])
            if not i+1>=len(bin2):
                final_time.append(bin2[i+1])
            i+=2


        #plt.scatter(np.arange(len(final_time)),final_time)
        #plt.show()

        for i in range(len(spectra)):
            spectra[i][j]=final_time[i]
    return spectra

#two simple functions for RFI removal (courtesy of H. Shetgaonkar)

def get_robust_mean_rms(input_arr, sigma_threshold):
    arr = np.copy(input_arr)
    ok = False
    iter_i, rms, mean = 0, 0.0, 0.0
    while not ok:
        iter_i += 1
        threshold = rms * sigma_threshold

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            mean = np.nanmean(arr)
        rms0 = rms

        if iter_i > 1:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                arr[abs(arr - mean) > threshold] = np.nan
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            rms = np.nanstd(arr)

        if iter_i > 1:
            if rms == 0.0:
                ok = True  # return
            elif np.isnan(rms):
                ok = True
            elif abs((rms0 / rms) - 1.0) < 0.01:
                ok = True
        print(f"mean={mean} ----> rms={rms}")
    return mean, rms

def remove_rfi(dynamic_spectrum, sigma_threshold=3):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        mean, rms = np.nanmean(dynamic_spectrum, axis=0), np.nanstd(dynamic_spectrum, axis=0)
    
    # option for robust mean/rms
    snr = 0.5 #float(np.sqrt(1))
    efficiency_x = mean / rms * snr
    
    mean_x, std_x = get_robust_mean_rms(efficiency_x, sigma_threshold)
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        flagged_ds = np.where(abs(mean_x - efficiency_x) > sigma_threshold * std_x, np.nan, dynamic_spectrum)

    return flagged_ds

## Load the Data

In [4]:
#Effelsberg data TELAMON
filename = "S20mm-SPECPOL-ARRAYDATA-1.fits"
telamon_data_fits = fits.open(filename)

## Let's inspect the data - See all the informations

In [5]:
#View FITS tables

In [6]:
#The data are stored in Table 1, so let's inspect that further and view the header info

In [14]:
#Let's extract some important information
scantime = 28 #scantime in seconds (not included in header)
bandwidth =
center_freq =
low_freq=
high_freq=

#and read out the data
data = 
#convert data from circular Polarization to StokesI (use getStokesI(data))
spectra = 

#extract some information
dt=
freqs=

## Data Vizualization & Basic Calibration


In [8]:
# Now that we have imported the data, let's plot it
# First, let's start with a simple waterfall/imshow plot

In [13]:
#Let's also look at the spectrum summed over time (use plotSpectrum(spectra,freqs))


#We need to correct for the bandpass (different sensitivy for different frequencies), let's do this with calibrateBandpass(spectra)

spectra=
#Now plot again the spectrum

In [10]:
#And the "lightcurve", summed over all frequencies can be plotted with plotLC(spectra)

#We have to remove a Noise Diode effect from the data, this can be done with removeNoiseCal(spectra)

#Now look at the lightcurve again to see the change

#And we can also remove the Blazar Target since we are not interested in it for this project (we are looking for transients!)
#To do this, we use removeNoiseCal(spectra,removeSource=True)

#Now look at the light curve again

In [11]:
#let's shift it to counts>0


#Let's look at the waterfaller/imshow plot again


# Clean RFI in the charts

In [16]:
# Let's try to get rid of the most prominent RFI components and clean the scan
# First, let's try a simple manual flagging where we exclude the brightest visible RFI channel
mask=np.zeros(spectra.shape)
mask[:,130:140]=1
spectra_masked=np.ma.masked_array(spectra,mask)

#Let's plot the LC again and the spectrum to see what changed


#Waterfall plot


In [17]:
#Still we see a lot of RFI-like signals, let's see if we can get rid of more of them if we put more masks

#plot lightcurve and spectrum again

#Waterfall plot

In [18]:
#Looks better but still not perfect, for instance if we zoom in a bit, we see there is still a lot of RFI signal
#-> all of this is quite tedious to get rid of manually, so we need some more powerful tools!

In [19]:
#Let's try a bit more sophisticated RFI removal using the functions introduced at the beginning 
#rms based flagging (remove_rfi(spectra))

#and plot lightcurve and spectrum


#Waterfall plot

# Let's look at some UIRAPURU data

In [23]:
#UIRAPURU data 
filename = "UIRAPURU_20230203_082000_59.fit"
uirapuru_data_fits = fits.open(filename)

#View FITS tables


#View Header Info
header=

#read out some basic information
date_obs=header["DATE-OBS"].replace("/","-")+"T"+header["TIME-OBS"]
date_end=header["DATE-END"].replace("/","-")+"T"+header["TIME-END"]
scantime=Time(date_end)-Time(date_obs)
scantime=scantime.sec #get scantime in seconds
dt=scantime/float(header["NAXIS1"])

header

In [25]:
image_data=np.transpose(uirapuru_data_fits[0].data)

"""
#let's try with a fake transient -> we play with this later
image_data[500:510,100:200]=image_data[500:510,100:200]+2
"""

#do waterfall plot

#plot spectrum and lightcurve
plotLC(image_data)
plotSpectrum(image_data,np.linspace(0.98,1.26,len(image_data[0])))

#try a bandpass calibration


#plot spectrum, lightcurve and waterfall again

"\n#let's try with a fake transient -> we play with this later\nimage_data[500:510,100:200]=image_data[500:510,100:200]+2\n"

In [26]:
# you can see, here it is quite challenging to recover any sensible data because of the complex RFI environment
#let's try with automatic rfi flagging...(remove_rfi(...))
cleaned_spectra=

#and plot lightcurve and spectrum

#Waterfall plot

# Effelsberg Data with a real astrophysical signal

In [27]:
#Effelsberg data TELAMON with Pulsar Data
filename = "S45mm-SPECPOL-ARRAYDATA-1.fits"
telamon_data_fits = fits.open(filename)

#extract info like before


#and read out the data

#convert data from circular Polarization to StokesI


#extract some information


# Now that we have imported the data, let's plot it
# First, let's start with a simple waterfall plot

In [28]:
#Let's do all the preliminary calibration steps we learned about before

#Calibrate the Bandpass


#Remove Noisecal


#shift spectrum to positive values
#let's shift it to counts>0


#Now plot again the Waterfaller 

In [29]:
#lets try some manual masking (try out different options)


#or lets try some automatic flagging


#Now plot again the Waterfaller 

In [30]:
#It's not perfect yet but now we can look at the Light Curve again between 10 and 20s (plotlim argument)


## Now let's take a look at a periodogram to identify the period of the pulsar

# How can we generalize this and detect not only periodic events but also transients (FRBs)?
There are a lot of pulsar analysis packages (mostly C-based) which use sophisticated algorhithms to tackle all of these problems. 

If we have time we can look at some of these programs in a virtual machine:
- PRESTO
- TransientX

Possible problems for the data sets we looked at today:
- different data formats -> need to convert the fits files to PSRFITS standard or filterbank files
- very bad time sampling in TELAMON and UIRAPURU data: usually pulsar data comes in much better time- (and frequency-) resolution, therefore some methods from the pulsar world might not be applicable to the data sets we looked at today!